In [98]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [99]:
# read data 
import pandas as pd 
df = pd.read_csv('same_value_at_all_t.csv')
t_set = df['t'].unique()
i_set = df['i'].unique()

buyer 

In [ ]:
# constraints 

# \sum_{i,t}Y_{i,t}<= Q^F_i 
def request(model):
    return sum(model.Y_it[t] for t in model.T) <= model.QF_i

#  f_{i,t}*g_{i,t} <= CO2_{i,t}
def carbon_target(model, t):  
    return model.f_it[t] * model.g_it[t] <= model.CO2_it[t]

# Y_{i,t} + g_{i,t} + Q^S_{i,t} = D_{i,t} 
def demand(model, t): 
    return (model.Y_it[t] + model.g_it[t] + model.QS_it[t] == model.D_it[t]) 

#  g_{i,t} <= \sum^t_{t'=1} h_{i,t'}
def generator_capacity (model, t):  
    return model.g_it[t] <= sum(model.h_it[tp] for tp in model.T if tp <= t)



In [ ]:
for i in i_set:
    # create a model 
    model = pyomo.ConcreteModel()

    # series 
    model.nt = pyomo.Param(initialize=len(df))
    model.T = pyomo.Set(initialize=t_set)

    # Parameters
    model.C_geninv_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i'] == i) & (df['t'] == t), 'C_geninv_it'].values[0] for t in t_set})  # C^{geninv}_{i,t}

    model.C_gen_it = pyomo.Param(model.T, initialize={t: df.loc[(df['i']==i)&(df['t']==t,'C_gen_it'])])        # C^{gen}_{i,t}
    model.f_it = pyomo.Param(model.T, initialize={t: val for t, val in zip(df['t'], df['f_it'])}[i])                # f_{i,t}
    model.CO2_it = pyomo.Param(model.T, initialize = {t: val for t, val in zip(df['t'], df['CO2_it'])}[i])          #CO2_{i,t}
    model.D_it = pyomo.Param(model.T, initialize = {t: val for t, val in zip(df['t'], df['D_it'])}[i])     #D_{i,t}
    model.PS_it = pyomo.Param(model.T, initialize ={t: val for t, val in zip(df['t'], df['PS_it'])}[i])    # P^S_{i,t}
    model.PF_i = pyomo.Param(initialize=1.0)                                                              # P^F_i = 5

    # Variables
    model.g_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)       # g_{i,t}
    model.h_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)       # h_{i,t}
    model.QS_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)      # Q^S_{i,t}
    model.Y_it = pyomo.Var(model.T, domain=pyomo.NonNegativeReals)       #Y_{i,t}
    model.QF_i = pyomo.Var(domain=pyomo.NonNegativeReals)                # Q^F_i

    # objective function 
    #min P^F_i * Q^F_i + \sum_t ( C^{geninv}_{i,t} * h_{i,t} + C^{gen}_{i,t} * g_{i,t} + P^S_{i,t} * Q^S_{i,t} )
    def objective_func(model):
        return model.PF_i * model.QF_i + \
            sum(model.C_geninv_it[t] * model.h_it[t] + model.C_gen_it[t] * model.g_it[t] \
                + model.PS_it[t] * model.QS_it[t] for t in model.T) 
    model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)

    model.requestConstraint = pyomo.Constraint(rule=request)
    model.carbontargetConstraint = pyomo.Constraint(model.T, rule=carbon_target)
    model.demandConstraint = pyomo.Constraint(model.T, rule=demand)
    model.generatorConstraint = pyomo.Constraint(model.T, rule=generator_capacity)

    result = solver.solve(model)

    print(result.solver.status)
    print(result.solver.termination_condition)

    print(model.objective())
    print(model.display())


KeyError: 0